In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
plotly.tools.set_credentials_file(username='vvr', api_key='Ker155UU70wk3Ccz6iyM')
import plotly.plotly as py
import plotly.graph_objs as go

color = sns.color_palette("husl", 8)

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer


%matplotlib inline

# import squarify

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
#from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
import re

from pandas import Series
from datetime import date

from scipy.sparse import coo_matrix, hstack


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train= pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data = pd.read_csv('campaign_data.csv')
submission = pd.read_csv('sample_submission_4fcZwvQ.csv')
submission1 = pd.read_csv('sample_submission_4fcZwvQ.csv')

In [3]:
submission2 = pd.read_csv('sample_submission_4fcZwvQ.csv')

In [4]:
submission3 = pd.read_csv('sample_submission_4fcZwvQ.csv')

In [5]:
submission4 = pd.read_csv('sample_submission_4fcZwvQ.csv')

In [6]:
submission5 = pd.read_csv('sample_submission_4fcZwvQ.csv')
submission6 = pd.read_csv('sample_submission_4fcZwvQ.csv')
submission7 = pd.read_csv('sample_submission_4fcZwvQ.csv')
submission8 = pd.read_csv('sample_submission_4fcZwvQ.csv')
submission9 = pd.read_csv('sample_submission_4fcZwvQ.csv')
submission10 = pd.read_csv('sample_submission_4fcZwvQ.csv')
submission11 = pd.read_csv('sample_submission_4fcZwvQ.csv')
submission12 = pd.read_csv('sample_submission_4fcZwvQ.csv')


In [7]:
train.columns  = ['id', 'userid', 'campaignid', 'send_date', 'open', 'click']

In [8]:
import re

data['subject'] = data['subject'].map(lambda x: re.sub(r'\W+', ' ', x))
data['email_body'] = data['email_body'].map(lambda x: re.sub(r'\W+', ' ', x))

data['subject'] = data['subject'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['email_body'] = data['email_body'].apply(lambda x: " ".join(x.lower() for x in x.split()))


def pir(data):
    t = data.text.str.split(expand=True).stack()
    return t.loc[t.str.len() >= 3].groupby(level=0).apply(' '.join)

data['subject'] = data['subject'].str.findall('\w{3,}').str.join(' ')
data['email_body'] = data['email_body'].str.findall('\w{3,}').str.join(' ')

In [9]:
import nltk
data['subject'] = data.apply(lambda row: nltk.word_tokenize(row['subject']), axis=1)
data['email_body'] = data.apply(lambda row: nltk.word_tokenize(row['email_body']), axis=1)


from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

data['subject'] = data["subject"].apply(lambda x: [stemmer.stem(y) for y in x])
data['email_body'] = data["email_body"].apply(lambda x: [stemmer.stem(y) for y in x])


from nltk.corpus import stopwords
stop = stopwords.words('english')

data['subject'] = data['subject'].apply(lambda x: ' '.join([word for word in x if word not in (stop)]))
data['email_body'] = data['email_body'].apply(lambda x: ' '.join([word for word in x if word not in (stop)]))

# TFIDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

data['subject'] = tfidf.fit_transform(data['subject']).toarray()
data['email_body'] = tfidf.fit_transform(data['email_body']).toarray()

# Label Encoding

In [11]:
le = LabelEncoder()
le.fit(data['communication_type'])

data['communication_type'] =  le.transform(data['communication_type'])

data.drop('email_url', axis=1, inplace = True)


# Concating

In [12]:
#print(train.head())

train['new_campain_id'] = train["campaignid"]
train['inddex'] = train.index

train.set_index('new_campain_id', inplace = True)

test['new_campain_id'] = test["campaign_id"]
test['inddex'] = test.index
test.set_index('campaign_id', inplace = True)
data.set_index('campaign_id', inplace = True)

train = pd.merge(train, data, left_index=True, right_index=True)

test = pd.merge(test, data, left_index=True, right_index=True)

In [13]:
train.set_index('inddex', inplace = True)
train.sort_index(inplace = True)


In [14]:
test.set_index('inddex', inplace = True)
test.sort_index(inplace = True)

# DateTime

In [15]:
train['send_date'] = pd.to_datetime(train['send_date'])

train['new_date_int'] = Series(train['send_date'].dt.date, dtype='datetime64')
train['new_date_int'] = train['new_date_int'].view('int64')


c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\pandas\core\internals.py:3224: FutureWarning:

Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.



In [16]:
test['send_date'] = pd.to_datetime(test['send_date'])

test['new_date_int'] = Series(test['send_date'].dt.date, dtype='datetime64')
test['new_date_int'] = test['new_date_int'].view('int64')

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\pandas\core\internals.py:3224: FutureWarning:

Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.



# Model Preparition

In [17]:
X_train = train[['userid', 'campaignid',
       'communication_type', 'total_links', 'no_of_internal_links',
       'no_of_images', 'no_of_sections', 'email_body', 'subject',
       'new_date_int']].copy()

In [18]:
y_train = train[['open', 'click']].copy()

In [19]:
X_test = test[['user_id', 'new_campain_id', 'communication_type',
       'total_links', 'no_of_internal_links', 'no_of_images', 'no_of_sections',
       'email_body', 'subject', 'new_date_int']]

# Hopefully Works Network

In [20]:
# cols_target = ['click', 'open']
cols_target = ['click']

In [21]:
X_train.isnull().sum()

userid                  0
campaignid              0
communication_type      0
total_links             0
no_of_internal_links    0
no_of_images            0
no_of_sections          0
email_body              0
subject                 0
new_date_int            0
dtype: int64

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=12.0)


y = train['click']
# train the model using X_dtm & y
logreg.fit(X_train, y)
# compute the training accuracy
#y_pred_X = logreg.predict(X_dtm)
#print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
# compute the predicted probabilities for X_test_dtm
test_y_prob = logreg.predict_proba(X_test)[:,1]
submission1['is_click'] = test_y_prob

In [23]:
submission1

,id,is_click
0,63_122715,0.011712
1,56_76206,0.011624
2,57_96189,0.011369
3,56_166917,0.011624
4,56_172838,0.011624
5,71_50253,0.011572
6,76_128244,0.011191
7,74_195513,0.011709
8,67_74617,0.010773
9,75_140599,0.011363


In [24]:
submission1 = submission.to_csv('submission1.csv',index=False)

# Hopefully works Ensemble

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
clf = RandomForestClassifier(max_depth=2, random_state=0)


y = train['click']
# train the model using X_dtm & y
clf.fit(X_train, y)
# compute the training accuracy
#y_pred_X = logreg.predict(X_dtm)
#print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
# compute the predicted probabilities for X_test_dtm
test_y_prob = clf.predict_proba(X_test)[:,1]
submission2['is_click'] = test_y_prob

In [26]:
submission2 = submission2.to_csv('submission2.csv',index=False)

# Another Ensemble

In [27]:
from sklearn.ensemble import AdaBoostClassifier

seed = 7
num_trees = 30
#kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
# results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
# print(results.mean())

model.fit(X_train, y)
# compute the training accuracy
#y_pred_X = logreg.predict(X_dtm)
#print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
# compute the predicted probabilities for X_test_dtm
test_y_prob = model.predict_proba(X_test)[:,1]
submission3['is_click'] = test_y_prob

In [28]:
submission3 = submission3.to_csv('submission3.csv',index=False)

In [29]:
from sklearn.ensemble import GradientBoostingClassifier
model2 = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)

model2.fit(X_train, y)
# compute the training accuracy
#y_pred_X = logreg.predict(X_dtm)
#print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
# compute the predicted probabilities for X_test_dtm
test_y_prob = model2.predict_proba(X_test)[:,1]
submission4['is_click'] = test_y_prob

In [30]:
submission4 = submission4.to_csv('submission4.csv',index=False)

# Genetic Algos

In [ ]:

from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2)

#tpot.fit(X_train, y_train)

#print(tpot.score(X_test, y_test))



y = train['click']
# train the model using X_dtm & y
tpot.fit(X_train, y)
# compute the training accuracy
#y_pred_X = logreg.predict(X_dtm)
#print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
# compute the predicted probabilities for X_test_dtm
test_y_prob = tpot.predict_proba(X_test)[:,1]
submission5['is_click'] = test_y_prob

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning:

Falling back to the python version of hypervolume module. Expect this to be very slow.

c:\users\admin\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:205: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

Optimization Progress:  30%|█████████████▉                                | 91/300 [3:07:11<12:02:49, 207.51s/pipeline]

In [ ]:
submission5 = submission5.to_csv('submission5.csv',index=False)

# Its DL time

In [ ]:
from keras.models import Model
from keras.models import Sequential

from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import TimeDistributed